# XGB Experiment based on 64 Variables 1911k Rows 14 Waves

In [1]:
%pwd

'/mnt/d/OneDrive - Kyushu University/ESG09_Article/Code'

In [2]:
%cd ..

/mnt/d/OneDrive - Kyushu University/ESG09_Article


/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Import Package

In [36]:
import numpy as np
import os 
import pandas as pd
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, RepeatedKFold
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb

## Load and Make Datasets

In [4]:
Df_Filename = os.path.join("Data", "GallupWB_Ml64var1911k14wave_v1.parquet")

In [5]:
Df = pd.read_parquet(Df_Filename)

In [6]:
Df.shape

(1911212, 64)

### Train and Test df

In [7]:
Df = Df.sample(frac=1, random_state=42).reset_index(drop=True)

In [8]:
Df['COUNTRY_ISO3'] = Df['COUNTRY_ISO3'].astype('category')

In [9]:
Df.head()

,wave,INCOME_2,Cantril_ladder,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,...,Corruption_government,Performance_leadership,Gender_female,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
0,10,35602.470662,7.0,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,64.0,1.0,5.0,0.0,2.0,3.0,NZL
1,11,20170.734058,5.0,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,18.0,1.0,6.0,0.0,2.0,1.0,LBN
2,6,8664.458086,4.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,15.0,1.0,6.0,1.0,3.0,3.0,BWA
3,12,30706.925977,2.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,33.0,2.0,1.0,1.0,2.0,1.0,KWT
4,7,4826.805232,8.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,29.0,8.0,1.0,1.0,3.0,4.0,HND


In [10]:
y = Df['Cantril_ladder']

In [11]:
X = Df.drop(columns=['Cantril_ladder'])

In [12]:
X.shape

(1911212, 63)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

### Basic XGBoost Experiment

In [14]:
model = xgb.XGBRegressor(objective='reg:squarederror', device = 'cuda', tree_method='hist', 
                         n_estimators=500, learning_rate=0.01, max_depth=5, 
                         random_state=42, enable_categorical=True  )
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [15]:
y_pred = model.predict(X_test)

/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [11:55:55] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [16]:
r2_score(y_test, y_pred) * 100

35.5966296143017

In [17]:
y_train_pred = model.predict(X_train)

In [18]:
r2_score(y_train, y_train_pred) * 100

35.76520229433162

### Fine-tune Hyperparameters

In [19]:
n_estimators_list = list(range(100, 1_100, 100))
learning_rate_list = [0.001, 0.01, 0.1]
max_depth_list = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
subsample_list = [0.6, 0.7, 0.8, 0.9, 1]

In [20]:
results_list = []
for n_estimators in n_estimators_list:
    for learning_rate in learning_rate_list:
        for max_depth in max_depth_list:
            for subsample in subsample_list:
                model = xgb.XGBRegressor(objective='reg:squarederror', device = 'cuda', 
                                         tree_method='hist', 
                                         n_estimators = n_estimators, 
                                         learning_rate = learning_rate, 
                                         max_depth = max_depth, 
                                         subsample = subsample,
                                         random_state=42, enable_categorical=True  )
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                test_r2 = r2_score(y_test, y_pred) * 100
                y_train_pred = model.predict(X_train)
                train_r2 = r2_score(y_train, y_train_pred) * 100
                row = [n_estimators, learning_rate, max_depth, 
                       subsample, train_r2, test_r2]
                print(row)
                results_list.append(row)

[100, 0.001, 3, 0.6, 4.415129833618048, 4.422782984695772]
[100, 0.001, 3, 0.7, 4.412602303237378, 4.420117210903651]
[100, 0.001, 3, 0.8, 4.410935930442783, 4.41855089772758]
[100, 0.001, 3, 0.9, 4.4092220231562, 4.417050344741524]
[100, 0.001, 3, 1, 4.4095473873662865, 4.417731229417187]
[100, 0.001, 4, 0.6, 4.903044187677407, 4.912279482273075]
[100, 0.001, 4, 0.7, 4.90045962560981, 4.909927152140847]
[100, 0.001, 4, 0.8, 4.897552002123485, 4.907206175979095]
[100, 0.001, 4, 0.9, 4.896992113351861, 4.906531296831296]
[100, 0.001, 4, 1, 4.897321347174199, 4.907313591677176]
[100, 0.001, 5, 0.6, 5.253012524011447, 5.257933675260096]
[100, 0.001, 5, 0.7, 5.250737347942824, 5.25565407777755]
[100, 0.001, 5, 0.8, 5.248080082113926, 5.253282434074924]
[100, 0.001, 5, 0.9, 5.247298210580098, 5.252559814047042]
[100, 0.001, 5, 1, 5.247236993414739, 5.253057381817328]
[100, 0.001, 6, 0.6, 5.504764814079033, 5.5036402645967435]
[100, 0.001, 6, 0.7, 5.502601052408529, 5.501884572701066]
[100, 

In [21]:
results_list_df = pd.DataFrame(results_list, columns = ['n_estimators', 'learning_rate', 'max_depth', 'subsample', 'train_r2', 'test_r2'])

In [22]:
results_list_df.to_parquet(os.path.join('Results', "Hyperparameter_TotalDatasetXgboost_ML.parquet"))

In [23]:
results_list_df.sort_values(by='test_r2', ascending=False).head(20)

,n_estimators,learning_rate,max_depth,subsample,train_r2,test_r2
2017,1000,0.01,14,0.8,73.276015,38.819217
2018,1000,0.01,14,0.9,73.793809,38.815904
2016,1000,0.01,14,0.7,72.578381,38.806969
2015,1000,0.01,14,0.6,71.789944,38.805366
2010,1000,0.01,13,0.6,62.664853,38.801609
2012,1000,0.01,13,0.8,63.720814,38.796181
2013,1000,0.01,13,0.9,64.127907,38.783612
2011,1000,0.01,13,0.7,63.262146,38.778143
1805,900,0.01,14,0.6,69.956337,38.764403
1807,900,0.01,14,0.8,71.463968,38.764091


### Fine tune other minor hyperparameters

In [24]:
gamma_list = [0, 0.1, 1, 10]
min_child_weight_list = [0, 0.1, 1, 10]
max_delta_step_list = [0, 0.1, 1, 10]
reg_lambda_list = [0, 0.1, 1, 10]
reg_alpha_list = [0, 0.1, 1, 10]

In [27]:
# 1000	0.01	14	0.8	73.276015	38.819217
results_list_otherhyper = []
for gamma in gamma_list:
    for min_child_weight in min_child_weight_list:
        for max_delta_step in max_delta_step_list:
            for reg_lambda in reg_lambda_list:
                for reg_alpha in reg_alpha_list:
                    model = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', 
                                             n_estimators = 1000,  learning_rate = 0.01, max_depth = 14,   subsample = 0.8,
                                             gamma = gamma, min_child_weight = min_child_weight, max_delta_step = max_delta_step,
                                             reg_lambda = reg_lambda, reg_alpha = reg_alpha, 
                                             random_state=42, enable_categorical=True  )
                    model.fit(X_train, y_train)
                    y_pred = model.predict(X_test)
                    test_r2 = r2_score(y_test, y_pred) * 100
                    y_train_pred = model.predict(X_train)
                    train_r2 = r2_score(y_train, y_train_pred) * 100
                    row = [gamma, min_child_weight, max_delta_step, reg_lambda, 
                           reg_alpha, train_r2, test_r2]
                    print(row)
                    model = None
                    ym_pred = None
                    ym_train_pred = None
                    results_list_otherhyper.append(row)

[0, 0, 0, 0, 0, 74.58133572083632, 38.81875538628259]
[0, 0, 0, 0, 0.1, 74.52356463079538, 38.80324844464739]
[0, 0, 0, 0, 1, 73.77540911029969, 38.865454782458684]
[0, 0, 0, 0, 10, 65.59706481001322, 38.833987826562]
[0, 0, 0, 0.1, 0, 74.33991654963579, 38.77196281917331]
[0, 0, 0, 0.1, 0.1, 74.29576741219832, 38.80584482581034]
[0, 0, 0, 0.1, 1, 73.56716859162312, 38.82750552230313]
[0, 0, 0, 0.1, 10, 65.59510278224437, 38.840330199992046]
[0, 0, 0, 1, 0, 73.27601463838486, 38.81921693243814]
[0, 0, 0, 1, 0.1, 73.19889984101783, 38.83919545358449]
[0, 0, 0, 1, 1, 72.58537780161689, 38.852988567597066]
[0, 0, 0, 1, 10, 65.1773119393791, 38.8460101018925]
[0, 0, 0, 10, 0, 68.17592362662849, 38.83103191716574]
[0, 0, 0, 10, 0.1, 68.17902379886345, 38.8663537582421]
[0, 0, 0, 10, 1, 67.65182287510775, 38.86324265148138]
[0, 0, 0, 10, 10, 62.248942548114904, 38.82000756994829]
[0, 0, 0.1, 0, 0, 26.905007835954965, 25.65514206133853]
[0, 0, 0.1, 0, 0.1, 26.37385521854343, 25.40432590984932

KeyboardInterrupt: 

### Bayes Search Hyperparameter (Test)

In [30]:
param_space = {
    'n_estimators': Integer(100, 1000),
    'learning_rate': Real(0.001, 0.1, prior='log-uniform'),
    'max_depth': Integer(3, 16),
    'subsample': Real(0.5, 1.0),
    'min_child_weight': Real(0.001, 10, prior='log-uniform'),
    'max_delta_step': Real(0.001, 10, prior='log-uniform'),
    'reg_lambda': Real(0.001, 10, prior='log-uniform'),
    'reg_alpha': Real(0.001, 10, prior='log-uniform'),
    'gamma': Real(0.001, 10, prior='log-uniform')
}

In [31]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True)

In [32]:
bayes_search = BayesSearchCV(
    estimator=xgb_reg,
    search_spaces=param_space,
    n_iter=20,
    scoring='r2',
    cv=5,
    n_jobs=10,
    verbose=2,
    random_state=42
)

In [33]:
bayes_search.fit(X, y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [14:32:58] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [14:32:58] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in t

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [14:33:15] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [14:33:16] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in t

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END gamma=2.236420282054271, learning_rate=0.058429282697611454, max_delta_step=0.016354619314682416, max_depth=15, min_child_weight=2.8609591832863424, n_estimators=156, reg_alpha=0.0035746551640890665, reg_lambda=0.025962684183737327, subsample=0.8178645509395852; total time=  17.0s
[CV] END gamma=1.7765766649807682, learning_rate=0.0022066991249460105, max_delta_step=0.24671088435225733, max_depth=1

/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [14:46:58] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END gamma=0.001, learning_rate=0.05934108113313268, max_delta_step=1.7155379671672653, max_depth=3, min_child_weight=0.001, n_estimators=1000, reg_alpha=10.0, reg_lambda=3.3562988535636995, subsample=0.5077999920334897; total time=  45.0s
[CV] END gamma=0.001, learning_rate=0.09999999999999999, max_delta_step=1.2503684029498734, max_depth=4, min_child_weight=10.0, n_estimators=1000, reg_alpha=0.001, reg_lambda=0.0026873611586649615, subsample=1.0; total time=  50.4s
[CV] END gamma=10.0, learning_rate=0.09999999999999999, max_delta_step=1.9026190989327987, max_depth=3, min_child_weight=10.0, n_estimators=334, reg_alpha=0.001, reg_lambda=0.001, subsample=0.8227007577862265; total time=  21.1s


/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:27:05] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:27:05] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in t

BayesSearchCV(cv=5,
              estimator=XGBRegressor(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=True, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None...
                             'min_child_weight': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'n_estimators': Integer(low=100, high=1000, prior='uniform', transform='normalize'),
                             'reg_alpha': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'reg_lambda': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'subsample': Real(low=0.5, high=1.0, prior='uniform', transform='normalize')},
              verbose=2)

In [34]:
CV_result = bayes_search.cv_results_

In [35]:
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gamma,param_learning_rate,param_max_delta_step,param_max_depth,param_min_child_weight,param_n_estimators,...,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
15,49.694564,0.135453,0.644396,0.068352,0.001,0.1,1.250368,4,10.0,1000,...,1.0,"{'gamma': 0.001, 'learning_rate': 0.0999999999...",0.375747,0.374343,0.373692,0.375783,0.376273,0.375168,0.000979,1
0,40.628238,0.040589,0.659970,0.071617,0.043693,0.02854,5.388551,7,0.479283,473,...,0.652232,"{'gamma': 0.04369339947510315, 'learning_rate'...",0.374639,0.373494,0.372273,0.374577,0.374843,0.373965,0.000968,2
8,34.293579,0.156735,0.475605,0.027637,6.636085,0.025191,3.066251,8,0.033439,290,...,0.797189,"{'gamma': 6.6360850776123375, 'learning_rate':...",0.371472,0.370205,0.368955,0.371285,0.371509,0.370685,0.000988,3
9,27.630808,0.086621,0.552814,0.067909,0.001034,0.043218,0.92285,6,0.206375,347,...,0.946919,"{'gamma': 0.0010340016434251913, 'learning_rat...",0.370928,0.369633,0.368478,0.370743,0.370781,0.370113,0.000939,4
10,43.213811,0.104432,0.573386,0.024014,0.001,0.1,10.0,3,0.001,1000,...,0.5,"{'gamma': 0.001, 'learning_rate': 0.0999999999...",0.368785,0.367462,0.366433,0.368457,0.368875,0.368002,0.000931,5
12,44.331663,0.210532,0.557530,0.054852,0.001,0.07582,6.968117,3,0.001,1000,...,0.56951,"{'gamma': 0.001, 'learning_rate': 0.0758200696...",0.368638,0.367105,0.366308,0.368362,0.368620,0.367807,0.000938,6
11,148.455940,3.286484,0.634548,0.179568,10.0,0.1,10.0,16,0.001,100,...,0.5,"{'gamma': 10.0, 'learning_rate': 0.09999999999...",0.368722,0.366784,0.365452,0.367377,0.368151,0.367297,0.001134,7
13,44.772206,0.095400,0.671334,0.129406,0.001,0.059341,1.715538,3,0.001,1000,...,0.5078,"{'gamma': 0.001, 'learning_rate': 0.0593410811...",0.367366,0.366057,0.364972,0.367360,0.367449,0.366641,0.000982,8
19,44.716399,0.128205,0.571311,0.056044,0.001,0.038965,10.0,3,0.001,1000,...,1.0,"{'gamma': 0.001, 'learning_rate': 0.0389652389...",0.364797,0.363662,0.362313,0.365305,0.364944,0.364204,0.001094,9
17,20.528313,0.068313,0.616276,0.057817,10.0,0.1,1.902619,3,10.0,334,...,0.822701,"{'gamma': 10.0, 'learning_rate': 0.09999999999...",0.364889,0.363329,0.362535,0.365009,0.365063,0.364165,0.001039,10


[CV] END gamma=0.001, learning_rate=0.03896523891612038, max_delta_step=10.0, max_depth=3, min_child_weight=0.001, n_estimators=1000, reg_alpha=0.001, reg_lambda=0.001, subsample=1.0; total time=  45.4s
[CV] END gamma=0.001, learning_rate=0.03896523891612038, max_delta_step=10.0, max_depth=3, min_child_weight=0.001, n_estimators=1000, reg_alpha=0.001, reg_lambda=0.001, subsample=1.0; total time=  45.1s
[CV] END gamma=0.001, learning_rate=0.03896523891612038, max_delta_step=10.0, max_depth=3, min_child_weight=0.001, n_estimators=1000, reg_alpha=0.001, reg_lambda=0.001, subsample=1.0; total time=  45.2s
[CV] END gamma=0.001, learning_rate=0.03896523891612038, max_delta_step=10.0, max_depth=3, min_child_weight=0.001, n_estimators=1000, reg_alpha=0.001, reg_lambda=0.001, subsample=1.0; total time=  45.4s
[CV] END gamma=0.001, learning_rate=0.03896523891612038, max_delta_step=10.0, max_depth=3, min_child_weight=0.001, n_estimators=1000, reg_alpha=0.001, reg_lambda=0.001, subsample=1.0; tota

### Bayes Search Hyperparameter (Test 3 times 1:9 CV)

In [37]:
param_space = {
    'n_estimators': Integer(100, 1000),
    'learning_rate': Real(0.001, 0.1, prior='log-uniform'),
    'max_depth': Integer(3, 16),
    'subsample': Real(0.5, 1.0),
    'min_child_weight': Real(0.001, 10, prior='log-uniform'),
    'max_delta_step': Real(0.001, 10, prior='log-uniform'),
    'reg_lambda': Real(0.001, 10, prior='log-uniform'),
    'reg_alpha': Real(0.001, 10, prior='log-uniform'),
    'gamma': Real(0.001, 10, prior='log-uniform')
}

In [38]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True)

In [45]:
bayes_search = BayesSearchCV(
    estimator=xgb_reg,
    search_spaces=param_space,
    n_iter=200,
    scoring='r2',
    cv=10,
    n_jobs=2,
    n_points = 1,
    verbose=2,
    random_state=42,
    return_train_score = True
)

In [46]:
bayes_search.fit(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [11:00:41] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [11:00:42] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in t

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=7, min_child_weight=0.47928274405969296, n_estimators=473, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time=  18.2s
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=7, min_child_weight=0.47928274405969296, n_estimators=473, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time=  18.7s
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=7, min_child_weight=0.47928274405969296, n_estimators=473, reg_alpha=

/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=7, min_child_weight=0.47928274405969296, n_estimators=473, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time=  18.7s
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=7, min_child_weight=0.47928274405969296, n_estimators=473, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time=  18.6s
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=7, min_child_weight=0.47928274405969296, n_estimators=473, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time=  18.9s
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=7, min_child_weight=0.47928

...

/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [18:31:11] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [18:31:31] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in t

[CV] END gamma=0.33209821575769133, learning_rate=0.0068639794182825545, max_delta_step=6.061126501381183, max_depth=15, min_child_weight=0.8955856146286302, n_estimators=976, reg_alpha=0.001079218593043752, reg_lambda=0.42561507255154285, subsample=0.5274166164117504; total time=22.5min
[CV] END gamma=0.33209821575769133, learning_rate=0.0068639794182825545, max_delta_step=6.061126501381183, max_depth=15, min_child_weight=0.8955856146286302, n_estimators=976, reg_alpha=0.001079218593043752, reg_lambda=0.42561507255154285, subsample=0.5274166164117504; total time=22.3min


/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END gamma=0.33209821575769133, learning_rate=0.0068639794182825545, max_delta_step=6.061126501381183, max_depth=15, min_child_weight=0.8955856146286302, n_estimators=976, reg_alpha=0.001079218593043752, reg_lambda=0.42561507255154285, subsample=0.5274166164117504; total time=22.9min
[CV] END gamma=0.33209821575769133, learning_rate=0.0068639794182825545, max_delta_step=6.061126501381183, max_depth=15, min_child_weight=0.8955856146286302, n_estimators=976, reg_alpha=0.001079218593043752, reg_lambda=0.42561507255154285, subsample=0.5274166164117504; total time=22.4min


KeyboardInterrupt: 

In [ ]:
CV_result = bayes_search.cv_results_

In [ ]:
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).head(10)